https://groups.google.com/forum/#!topic/recsys-challenge2020/piIOgvhQZFk

    We are observing strange patterns in the datasets and are wondering whether they are artificial or real. We are concerned about using features that may be artificial in our models. These features may increase local validation and increase public leaderboard but they may not predict the final test dataset.

    (Here is a hypothetical example. If all pseudo negatives are chosen from English language tweets, then there will be an unnatural pattern showing that English tweets have more negative targets. And if training data pseudo negatives are selected from English language tweets and test data pseudo negatives are chosen from Spanish language tweets, that will create a problem).


    I want to second this, I created some "dummy features" which makes NO SENSE AT ALL (think of features such as "Was it 35 celsius the day the tweet was made?"), and it got a huge improvement both on CV and Leaderboard. I think its clear something is not right, or I have no understanding how twitter works.


# start

In [1]:
trntmstmp=1584412344
valtmstmp=1585569704

import datetime
def showtm(tm): return datetime.datetime.fromtimestamp(tm).strftime('%Y-%m-%d %H:%M:%S')
print([showtm(tm) for tm in (trntmstmp, valtmstmp)])


nsmpl=int(1e5)

['2020-03-17 02:32:24', '2020-03-30 12:01:44']


# setup

In [2]:
grand_total=int(1.5e8)
valid_total=int(1.4e7)
print(f'trnsz {grand_total:,}, valsz {valid_total:,}')
tm_trn_min,tm_trn_max=(1580947200, 1581551999)
tm_val_min,tm_val_max=(1581552000, 1582156799)
print([showtm(tm) for tm in (tm_trn_min,tm_trn_max)])
print([showtm(tm) for tm in (tm_val_min,tm_val_max)])

trnsz 150,000,000, valsz 14,000,000
['2020-02-06 00:00:00', '2020-02-12 23:59:59']
['2020-02-13 00:00:00', '2020-02-19 23:59:59']


In [3]:
from tqdm import tqdm
from pathlib import Path
from collections import Counter
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, auc, precision_recall_curve
import pickle
import xgboost as xgb
import lightgbm as lgb

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)


import datetime
def dtnow(): return datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

def iou(s1,s2):
    return len(s1&s2) / len(s1|s2)


SEED=101
HOME='/data/git/recsys20'
p_in=f'{HOME}/input'
# p_out=f'{HOME}/output/{PRFX}'
# Path(p_out).mkdir(exist_ok=True, parents=True)

from sklearn.metrics import precision_recall_curve, auc, log_loss

def compute_prauc(pred, gt):
    prec, recall, thresh = precision_recall_curve(gt, pred)
    prauc = auc(recall, prec)
    return prauc

def calculate_ctr(gt):
    positive = len([x for x in gt if x == 1])
    ctr = positive/float(len(gt))
    return ctr

def compute_rce(pred, gt):
    cross_entropy = log_loss(gt, pred)
    data_ctr = calculate_ctr(gt)
    strawman_cross_entropy = log_loss(gt, [data_ctr for _ in range(len(gt))])
    return (1.0 - cross_entropy/strawman_cross_entropy)*100.0

cols=['toks','hshtgs','twtid','media','links','domns','twttyp','lang','tm','u1id','u1_fllwer_cnt','u1_fllwing_cnt','u1_vrfed','u1_create_tm','u2id','u2_follower_cnt','u2_following_cnt','u2_vrfed','u2_create_tm','u1_fllw_u2','reply_tm','retwt_tm','retwt_cmmnt_tm','like_tm',]

cols_val = cols[:-4]
cols_tgt_tmstmp=[ 'retwt_tm', 'reply_tm', 'like_tm', 'retwt_cmmnt_tm',]
cols_tgt=[o.split('_tm')[0] for o in cols_tgt_tmstmp]
tgts             = ['Retweet','Reply','Like','RTwCmnt',]
assert cols_tgt == ['retwt',  'reply','like','retwt_cmmnt',]
ntgts=len(tgts)



# prep

In [4]:
%%time
dfval = pd.read_csv(f'{p_in}/val_{valtmstmp}.tsv',sep='\x01',header=None,names=cols_val,nrows=nsmpl)
lndfval=len(dfval)
print(f'{lndfval:,}')

100,000
CPU times: user 678 ms, sys: 95.1 ms, total: 773 ms
Wall time: 1.14 s


In [5]:
%%time
df = pd.read_csv(f'{p_in}/trn_{trntmstmp}.tsv',sep='\x01',header=None,names=cols,nrows=nsmpl)
lndf=len(df)
print(f'{lndf:,}')

100,000
CPU times: user 704 ms, sys: 83.2 ms, total: 787 ms
Wall time: 1.09 s


In [6]:
df.columns

Index(['toks', 'hshtgs', 'twtid', 'media', 'links', 'domns', 'twttyp', 'lang',
       'tm', 'u1id', 'u1_fllwer_cnt', 'u1_fllwing_cnt', 'u1_vrfed',
       'u1_create_tm', 'u2id', 'u2_follower_cnt', 'u2_following_cnt',
       'u2_vrfed', 'u2_create_tm', 'u1_fllw_u2', 'reply_tm', 'retwt_tm',
       'retwt_cmmnt_tm', 'like_tm'],
      dtype='object')

In [8]:
df[cols_tgt]=df[cols_tgt_tmstmp].notna()
df['enged']=df[cols_tgt].sum(1).astype(bool)

In [73]:
df['dt']=pd.to_datetime(df.tm, unit='s')

df['dt_sec']=df.dt.dt.second
df['dt_min']=df.dt.dt.minute
df['dt_day']=df.dt.dt.day

In [83]:
df.groupby('dt_min').enged.mean()

dt_min
0     0.419781
1     0.499202
2     0.482721
3     0.526844
4     0.510890
5     0.509516
6     0.516697
7     0.502861
8     0.506276
9     0.524717
10    0.512223
11    0.518088
12    0.517610
13    0.523057
14    0.507201
15    0.509793
16    0.525921
17    0.514054
18    0.496134
19    0.523529
20    0.496770
21    0.514864
22    0.512940
23    0.521378
24    0.523515
25    0.528152
26    0.521512
27    0.495402
28    0.526452
29    0.513651
30    0.456625
31    0.538879
32    0.508277
33    0.532071
34    0.498082
35    0.511740
36    0.536802
37    0.510743
38    0.512524
39    0.513932
40    0.493333
41    0.510137
42    0.533999
43    0.512255
44    0.528615
45    0.509977
46    0.528031
47    0.536706
48    0.510076
49    0.535128
50    0.510961
51    0.509114
52    0.500966
53    0.513593
54    0.530136
55    0.519347
56    0.507405
57    0.543915
58    0.517086
59    0.525478
Name: enged, dtype: float64

In [23]:
cols_tgt

['retwt', 'reply', 'like', 'retwt_cmmnt']

In [96]:
display(df.toks[df.enged].sample(2).values)
display(df.toks[~df.enged].sample(2).values)

array(['101\t1894\t5952\t3504\t1912\t35331\t13913\t18226\t25053\t6211\t2932\t1939\t3197\t2149\t55627\t20885\t28100\t1895\t119\t1831\t27058\t19973\t11588\t5986\t1916\t32663\t8272\t2104\t119\t1797\t4004\t3104\t1906\t2149\t4671\t1943\t8272\t2104\t10059\t2002\t20703\t22396\t16457\t11312\t29942\t7418\t2625\t3099\t5752\t1980\t4307\t2380\t1882\t119\t1832\t54643\t11668\t39004\t12434\t22946\t3984\t5630\t1980\t6033\t102901\t119\t1797\t95397\t34922\t10634\t88742\t11281\t2068\t1985\t92240\t39004\t36394\t11377\t4368\t5876\t84543\t1882\t119\t1833\t3460\t2179\t4305\t1939\t4898\t1965\t1881\t3393\t7069\t1943\t4388\t8137\t1980\t2583\t1915\t1882\t119\t1797\t3906\t3820\t4898\t3388\t1947\t6195\t4352\t1915\t1882\t2207\t7229\t1946\t4653\t7191\t1943\t4388\t8137\t1980\t4037\t2079\t1882\t102',
       '101\t102545\t11223\t10380\t10452\t89025\t10189\t19220\t125\t82808\t17367\t10114\t10347\t22021\t59648\t11912\t119\t119\t24781\t10301\t17446\t82808\t11049\t44346\t56996\t10108\t35691\t10453\t98800\t10426\t136\t119\t

array(['101\t1994\t53669\t13913\t35412\t53669\t29442\t2256\t7107\t63928\t27849\t56910\t18767\t18825\t4180\t17984\t27607\t3397\t2227\t10096\t119539\t119539\t1890\t6691\t3832\t1936\t18767\t67946\t1891\t10073\t3596\t1881\t95154\t22445\t95977\t5712\t3175\t3390\t33469\t25204\t8406\t100\t14120\t131\t120\t120\t188\t119\t11170\t120\t141\t10162\t10929\t10686\t11305\t11537\t10174\t11733\t10369\t102',
       '101\t11741\t11972\t169\t137\t10406\t50201\t28883\t13091\t12096\t10220\t10493\t10182\t78036\t11639\t61643\t11861\t11181\t119\t11818\t28086\t12014\t119\t100\t102'],
      dtype=object)

In [67]:
display(df.u1id[df.enged].sample(5))
display(df.u1id[~df.enged].sample(5))

10315    C6758D692A850E4C67B2763B66D1CFA8
93739    B417D5255878BA3DA4EF1F069EAFFD4C
41433    D88CA2D9CFCBE92F8F973B0DEF8AD857
54513    AE3536130266CB89AB16E21DF82C7582
51003    5B16FB129F85DC61F805751C0EAE0169
Name: u1id, dtype: object

48542    1631D84CF3250668B24AFC7EEFBDB969
80587    4F1B8367D779BB513BEEFD9295645D64
66636    808FD2641D7B10A1E4FA725127AE6D93
25427    913B18B45CE33B8393C9BA6303A727CF
71353    12CB4F6B77EF6A0379AEE0C15F8AE828
Name: u1id, dtype: object

In [71]:
display(df.u2id[df.enged].sample(5))
display(df.u2id[~df.enged].sample(5))

12719    002CA84459C93F47C36992D852FC7D0E
5050     0011A94FED15AE34ACE7EDDA3D9A6C6D
97536    01599A01F73D4F37E67CF13DFF4234EC
54626    00C1A4E03D051DB746AF621F0D3C4E4F
7621     001A86A799F0EDFCE0EBF256F4003DE2
Name: u2id, dtype: object

42578    0096373599FEF023116C2EE5D43481DE
49998    00B0F8894F67AF1D5AE52B84FD473FC5
77778    01132D1F6B74AB08667B1943B5DE8EA6
48553    00AB9B9536492D7550C2A91382A3D77D
62844    00DE43F0C93DF5D96A862BD4FF0AAB52
Name: u2id, dtype: object

In [98]:
set_u1 = set(df.u1id)
set_u2 = set(df.u2id)

In [101]:
df['u1inu2']=df.u1id.isin(set_u2)
df['u2inu1']=df.u2id.isin(set_u1)

In [108]:
df.groupby('u1inu2').enged.mean()

u1inu2
False    0.510655
True     0.673797
Name: enged, dtype: float64

In [109]:
df.groupby('u2inu1').enged.mean()

u2inu1
False    0.510156
True     0.734637
Name: enged, dtype: float64

In [110]:
df.groupby('u1inu2').like.mean()

u1inu2
False    0.439071
True     0.556150
Name: like, dtype: float64

In [111]:
df.groupby('u1inu2').reply.mean()

u1inu2
False    0.027481
True     0.058824
Name: reply, dtype: float64

In [112]:
df.groupby('u2inu1').reply.mean()

u2inu1
False    0.027458
True     0.050279
Name: reply, dtype: float64

In [113]:
df.groupby('u2inu1').reply.mean()

u2inu1
False    0.027458
True     0.050279
Name: reply, dtype: float64

In [28]:
dict(df.groupby('lang').size())

{'022EC308651FACB02794A8147AEE1B78': 2361,
 '0331BF70E606D62D92C96CE9AD71A7CF': 84,
 '06BEAB41D66CCFF329D1ED8BA120A6C2': 68,
 '06D61DCBBE938971E1EA0C38BD9B5446': 8887,
 '125C57F4FA6D4E110983FB11B52EFD4E': 2410,
 '167115458A0DBDFF7E9C0C53A83BAC9B': 3161,
 '190BA7DA361BC06BC1D7E824C378064D': 230,
 '1BC639981AE88E09129594B11F894A21': 8,
 '1FFD2FE4297F5E70EBC6C3230D95CB9C': 173,
 '2216D01F7B48554E4211021A46816FCF': 12,
 '22C448FF81263D4BAF2A176145EE9EAD': 16127,
 '259A6F6DFD672CB1F883CBEC01B99F2D': 18,
 '2996EB2FE8162C076D070A4C8D6532CD': 916,
 '3121F7240D488F74EEED9312E174B217': 21,
 '3820C29CBCA409A33BADF68852057C4A': 515,
 '3A85BCEC571C3F5AB1069E4924189177': 24,
 '3E16B11B7ADE3A22DDFC4423FBCEAD5D': 568,
 '4249CE88433AEA3F8DCEECF008B3CB95': 30,
 '4690215948DBF6872B8ED1C2BC87B17E': 1,
 '48236EC80FDDDFADE99420ABC9210DDF': 173,
 '4DC22C3F31C5C43721E6B5815A595ED6': 3288,
 '54208B51D44E7D91DC2F3DD02ADEDEC2': 20,
 '544FA32458C903F1125FE6598300A047': 30,
 '57ADD4576E2AD6648E9B2DE32F3462A5': 6,


In [29]:
dict(df.groupby('lang').like.mean())

{'022EC308651FACB02794A8147AEE1B78': 0.23252858958068615,
 '0331BF70E606D62D92C96CE9AD71A7CF': 0.5119047619047619,
 '06BEAB41D66CCFF329D1ED8BA120A6C2': 0.6470588235294118,
 '06D61DCBBE938971E1EA0C38BD9B5446': 0.3974344548216496,
 '125C57F4FA6D4E110983FB11B52EFD4E': 0.44481327800829873,
 '167115458A0DBDFF7E9C0C53A83BAC9B': 0.4194875039544448,
 '190BA7DA361BC06BC1D7E824C378064D': 0.44782608695652176,
 '1BC639981AE88E09129594B11F894A21': 0.875,
 '1FFD2FE4297F5E70EBC6C3230D95CB9C': 0.4277456647398844,
 '2216D01F7B48554E4211021A46816FCF': 0.75,
 '22C448FF81263D4BAF2A176145EE9EAD': 0.46859304272338315,
 '259A6F6DFD672CB1F883CBEC01B99F2D': 0.3333333333333333,
 '2996EB2FE8162C076D070A4C8D6532CD': 0.49672489082969434,
 '3121F7240D488F74EEED9312E174B217': 0.6190476190476191,
 '3820C29CBCA409A33BADF68852057C4A': 0.516504854368932,
 '3A85BCEC571C3F5AB1069E4924189177': 0.7083333333333334,
 '3E16B11B7ADE3A22DDFC4423FBCEAD5D': 0.6584507042253521,
 '4249CE88433AEA3F8DCEECF008B3CB95': 0.366666666666666